---
## 1. Teoretický Úvod

### 1.1 Hyperparameters vs Parameters

| Typ | Popis | Příklady |
|-----|-------|----------|
| **Parameters** | Naučené během trénování | Váhy stromů, split thresholds |
| **Hyperparameters** | Nastavené před tréninkem | n_estimators, max_depth |

### 1.2 Grid Search

Systematické prohledávání všech kombinací hyperparametrů:

$$\text{Total combinations} = \prod_{i=1}^{n} |H_i|$$

kde $|H_i|$ je počet hodnot pro i-tý hyperparametr.

### 1.3 TimeSeriesSplit

Pro finanční data **MUSÍME** používat TimeSeriesSplit místo standardní K-Fold CV:

```
Fold 1: [Train: ████████] [Test: ██]
Fold 2: [Train: ██████████] [Test: ██]
Fold 3: [Train: ████████████] [Test: ██]
Fold 4: [Train: ██████████████] [Test: ██]
Fold 5: [Train: ████████████████] [Test: ██]
```

**Proč?**
- Zachovává chronologické pořadí
- Zabraňuje data leakage
- Simuluje reálné použití (trénujeme na minulosti, predikujeme budoucnost)

---
## 2. Setup Prostředí

In [ ]:
# Instalace (pro Colab)
!pip install pandas numpy scikit-learn joblib matplotlib seaborn tqdm -q

print("✓ Knihovny nainstalovány")

In [ ]:
# Import knihoven
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from tqdm.notebook import tqdm
import warnings
import os
import joblib
import json

# Scikit-learn
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingClassifier
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import (
    TimeSeriesSplit, GridSearchCV, RandomizedSearchCV, cross_val_score
)
from sklearn.metrics import (
    mean_absolute_error, mean_squared_error, r2_score,
    accuracy_score, f1_score, make_scorer
)

warnings.filterwarnings('ignore')
np.random.seed(42)

print("✓ Knihovny načteny")

In [ ]:
# Připojení Google Drive
try:
    from google.colab import drive
    drive.mount('/content/drive')
    DRIVE_PATH = '/content/drive/MyDrive/MachineLearning'
    RUNNING_ON_COLAB = True
    print(f"✓ Google Drive připojen: {DRIVE_PATH}")
except:
    DRIVE_PATH = '.'
    RUNNING_ON_COLAB = False
    print("ℹ️ Lokální prostředí")

# Cesty
DATA_PATH = f"{DRIVE_PATH}/data"
MODEL_PATH = f"{DRIVE_PATH}/models"
os.makedirs(MODEL_PATH, exist_ok=True)

---
## 3. Načtení Dat

In [ ]:
# Načtení kompletního datasetu
complete_path = f"{DATA_PATH}/complete/all_sectors_complete_10y.csv"
df = pd.read_csv(complete_path, parse_dates=['date'])

print(f"📈 Dataset načten: {len(df):,} záznamů")

In [ ]:
# Definice features
FEATURE_COLS = [
    'open', 'high', 'low', 'close', 'volume',
    'returns', 'volatility_12m', 'rsi_14',
    'macd', 'macd_signal', 'macd_hist',
    'sma_3', 'sma_6', 'sma_12',
    'ema_3', 'ema_6', 'ema_12',
    'volume_change', 'price_momentum'
]

FUNDAMENTAL_COLS = [
    'PE', 'PB', 'PS', 'EV_EBITDA',
    'ROE', 'ROA', 'Profit_Margin',
    'Debt_to_Equity', 'Current_Ratio',
    'Revenue_Growth_YoY', 'Earnings_Growth_YoY'
]

# Dostupné features
available_features = [f for f in FEATURE_COLS if f in df.columns]
available_fundamentals = [f for f in FUNDAMENTAL_COLS if f in df.columns]

print(f"📊 Features: {len(available_features)}")
print(f"📊 Fundamentals: {len(available_fundamentals)}")

In [ ]:
# Příprava dat pro tuning

# 1. Pro Fundamental Predictor (RF Regressor)
regressor_df = df.dropna(subset=available_features + available_fundamentals)
X_reg = regressor_df[available_features].values
y_reg = regressor_df[available_fundamentals].values

print(f"📊 Regressor Data: {X_reg.shape[0]:,} samples, {X_reg.shape[1]} features → {y_reg.shape[1]} targets")

In [ ]:
# 2. Pro Price Classifier
THRESHOLD = 0.03

# Vytvoření target
classifier_df = df.copy()
classifier_df['future_return'] = classifier_df.groupby('ticker')['close'].shift(-1) / classifier_df['close'] - 1

conditions = [
    classifier_df['future_return'] < -THRESHOLD,
    classifier_df['future_return'] > THRESHOLD,
]
choices = [0, 2]
classifier_df['target'] = np.select(conditions, choices, default=1)

# Čištění
all_features = available_features + available_fundamentals
classifier_df = classifier_df.dropna(subset=all_features + ['target'])

X_clf = classifier_df[all_features].values
y_clf = classifier_df['target'].values.astype(int)

print(f"📊 Classifier Data: {X_clf.shape[0]:,} samples, {X_clf.shape[1]} features → 3 classes")

---
## 4. TimeSeriesSplit Konfigurace

In [ ]:
# TimeSeriesSplit konfigurace
N_SPLITS = 5

tscv = TimeSeriesSplit(n_splits=N_SPLITS)

# Vizualizace splitů
print(f"📊 TimeSeriesSplit s {N_SPLITS} foldy")
print("="*60)

for i, (train_idx, test_idx) in enumerate(tscv.split(X_clf)):
    train_pct = len(train_idx) / len(X_clf) * 100
    test_pct = len(test_idx) / len(X_clf) * 100
    
    train_bar = '█' * int(train_pct / 2)
    test_bar = '▓' * int(test_pct / 2)
    
    print(f"Fold {i+1}: {train_bar}{test_bar} (Train: {len(train_idx):,} | Test: {len(test_idx):,})")

---
## 5. Grid Search pro RF Regressor (Fundamental Predictor)

### 5.1 Parametrový Prostor

| Parametr | Hodnoty | Popis |
|----------|---------|-------|
| `n_estimators` | [100, 200, 300] | Počet stromů |
| `max_depth` | [10, 15, 20, None] | Maximální hloubka |
| `min_samples_split` | [2, 5, 10] | Min. samples pro split |
| `min_samples_leaf` | [1, 2, 4] | Min. samples v listu |

In [ ]:
# Parametrový prostor pro RF Regressor
rf_reg_param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 15, 20],
    'min_samples_split': [5, 10],
    'min_samples_leaf': [2, 4]
}

# Počet kombinací
n_combinations = np.prod([len(v) for v in rf_reg_param_grid.values()])
print(f"🔍 Grid Search pro RF Regressor")
print(f"   Parametrů: {len(rf_reg_param_grid)}")
print(f"   Kombinací: {n_combinations}")
print(f"   Celkem fitů: {n_combinations * N_SPLITS}")

In [ ]:
%%time

# Pro úsporu času použijeme jen jeden target (P/E)
# V praxi by se optimalizovalo pro všechny

pe_idx = available_fundamentals.index('PE') if 'PE' in available_fundamentals else 0
y_reg_pe = y_reg[:, pe_idx]

# Standardizace
scaler_reg = StandardScaler()
X_reg_scaled = scaler_reg.fit_transform(X_reg)

# Grid Search
print("🚀 Spouštím Grid Search pro RF Regressor (P/E target)...")

rf_reg = RandomForestRegressor(random_state=42, n_jobs=-1)

grid_search_reg = GridSearchCV(
    estimator=rf_reg,
    param_grid=rf_reg_param_grid,
    cv=tscv,
    scoring='neg_mean_absolute_error',
    n_jobs=-1,
    verbose=1
)

grid_search_reg.fit(X_reg_scaled, y_reg_pe)

print("\n✅ Grid Search dokončen!")

In [ ]:
# Výsledky Grid Search pro Regressor
print("📊 VÝSLEDKY - RF REGRESSOR")
print("="*60)
print(f"\n🏆 Nejlepší parametry:")
for param, value in grid_search_reg.best_params_.items():
    print(f"   {param}: {value}")

print(f"\n📊 Nejlepší MAE: {-grid_search_reg.best_score_:.4f}")

# Top 5 kombinací
results_reg = pd.DataFrame(grid_search_reg.cv_results_)
results_reg = results_reg.sort_values('rank_test_score')

print(f"\n📊 Top 5 kombinací:")
print("-"*60)
for i, row in results_reg.head(5).iterrows():
    print(f"   Rank {row['rank_test_score']}: MAE={-row['mean_test_score']:.4f} ± {row['std_test_score']:.4f}")
    print(f"      params: {row['params']}")

---
## 6. Grid Search pro RF Classifier (Price Classifier)

In [ ]:
# Parametrový prostor pro RF Classifier
rf_clf_param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [8, 12, 16],
    'min_samples_split': [5, 10],
    'min_samples_leaf': [2, 5],
    'class_weight': ['balanced', None]
}

n_combinations_clf = np.prod([len(v) for v in rf_clf_param_grid.values()])
print(f"🔍 Grid Search pro RF Classifier")
print(f"   Kombinací: {n_combinations_clf}")
print(f"   Celkem fitů: {n_combinations_clf * N_SPLITS}")

In [ ]:
%%time

# Standardizace
scaler_clf = StandardScaler()
X_clf_scaled = scaler_clf.fit_transform(X_clf)

# Grid Search
print("🚀 Spouštím Grid Search pro RF Classifier...")

rf_clf = RandomForestClassifier(random_state=42, n_jobs=-1)

grid_search_clf = GridSearchCV(
    estimator=rf_clf,
    param_grid=rf_clf_param_grid,
    cv=tscv,
    scoring='f1_weighted',
    n_jobs=-1,
    verbose=1
)

grid_search_clf.fit(X_clf_scaled, y_clf)

print("\n✅ Grid Search dokončen!")

In [ ]:
# Výsledky Grid Search pro Classifier
print("📊 VÝSLEDKY - RF CLASSIFIER")
print("="*60)
print(f"\n🏆 Nejlepší parametry:")
for param, value in grid_search_clf.best_params_.items():
    print(f"   {param}: {value}")

print(f"\n📊 Nejlepší F1-Score: {grid_search_clf.best_score_:.4f}")

# Top 5 kombinací
results_clf = pd.DataFrame(grid_search_clf.cv_results_)
results_clf = results_clf.sort_values('rank_test_score')

print(f"\n📊 Top 5 kombinací:")
print("-"*60)
for i, row in results_clf.head(5).iterrows():
    print(f"   Rank {row['rank_test_score']}: F1={row['mean_test_score']:.4f} ± {row['std_test_score']:.4f}")
    print(f"      params: {row['params']}")

---
## 7. Porovnání s Gradient Boosting

In [ ]:
# Porovnání RF vs Gradient Boosting pro klasifikaci

gb_param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.05, 0.1],
    'min_samples_split': [5, 10]
}

print("🚀 Grid Search pro Gradient Boosting Classifier...")

gb_clf = GradientBoostingClassifier(random_state=42)

# Menší sample pro rychlost
sample_size = min(5000, len(X_clf_scaled))
sample_idx = np.random.choice(len(X_clf_scaled), sample_size, replace=False)
sample_idx = np.sort(sample_idx)  # Zachovat chronologické pořadí

X_sample = X_clf_scaled[sample_idx]
y_sample = y_clf[sample_idx]

In [ ]:
%%time

grid_search_gb = GridSearchCV(
    estimator=gb_clf,
    param_grid=gb_param_grid,
    cv=TimeSeriesSplit(n_splits=3),
    scoring='f1_weighted',
    n_jobs=-1,
    verbose=1
)

grid_search_gb.fit(X_sample, y_sample)

print("\n✅ Grid Search GB dokončen!")

In [ ]:
# Porovnání RF vs GB
print("📊 POROVNÁNÍ ALGORITMŮ")
print("="*60)
print(f"\n{'Algoritmus':<25} {'Nejlepší F1':>15} {'Nejlepší Params'}")
print("-"*60)
print(f"{'Random Forest':<25} {grid_search_clf.best_score_:>15.4f}")
print(f"{'Gradient Boosting':<25} {grid_search_gb.best_score_:>15.4f}")

# Vítěz
if grid_search_clf.best_score_ > grid_search_gb.best_score_:
    print(f"\n🏆 Vítěz: Random Forest")
else:
    print(f"\n🏆 Vítěz: Gradient Boosting")

---
## 8. Vizualizace Výsledků

In [ ]:
# Vizualizace výsledků Grid Search
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. RF Regressor - vliv n_estimators
ax1 = axes[0, 0]
for depth in rf_reg_param_grid['max_depth']:
    mask = results_reg['param_max_depth'] == depth
    subset = results_reg[mask].groupby('param_n_estimators')['mean_test_score'].mean()
    ax1.plot(subset.index, -subset.values, marker='o', label=f'depth={depth}')
ax1.set_xlabel('n_estimators')
ax1.set_ylabel('MAE')
ax1.set_title('RF Regressor: MAE vs n_estimators', fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. RF Classifier - vliv n_estimators
ax2 = axes[0, 1]
for depth in rf_clf_param_grid['max_depth']:
    mask = results_clf['param_max_depth'] == depth
    subset = results_clf[mask].groupby('param_n_estimators')['mean_test_score'].mean()
    ax2.plot(subset.index, subset.values, marker='o', label=f'depth={depth}')
ax2.set_xlabel('n_estimators')
ax2.set_ylabel('F1-Score')
ax2.set_title('RF Classifier: F1 vs n_estimators', fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. Heatmap - RF Classifier (n_estimators vs max_depth)
ax3 = axes[1, 0]
pivot = results_clf.pivot_table(
    values='mean_test_score',
    index='param_max_depth',
    columns='param_n_estimators',
    aggfunc='mean'
)
sns.heatmap(pivot, annot=True, fmt='.3f', cmap='YlGnBu', ax=ax3)
ax3.set_title('RF Classifier: F1 Heatmap', fontweight='bold')

# 4. CV Score distribuce
ax4 = axes[1, 1]
ax4.boxplot([results_reg['mean_test_score'] * -1, results_clf['mean_test_score']],
            labels=['Regressor (MAE)', 'Classifier (F1)'])
ax4.set_ylabel('Score')
ax4.set_title('Distribuce CV Score', fontweight='bold')

plt.tight_layout()
plt.savefig(f"{DATA_PATH}/hyperparameter_tuning.png", dpi=150, bbox_inches='tight')
plt.show()

print(f"\n💾 Graf uložen: {DATA_PATH}/hyperparameter_tuning.png")

---
## 9. Uložení Optimálních Parametrů

In [ ]:
# Uložení optimálních parametrů
optimal_params = {
    'fundamental_predictor': {
        'best_params': grid_search_reg.best_params_,
        'best_score': float(-grid_search_reg.best_score_),
        'metric': 'MAE'
    },
    'price_classifier': {
        'best_params': grid_search_clf.best_params_,
        'best_score': float(grid_search_clf.best_score_),
        'metric': 'F1-weighted'
    },
    'gradient_boosting': {
        'best_params': grid_search_gb.best_params_,
        'best_score': float(grid_search_gb.best_score_),
        'metric': 'F1-weighted'
    },
    'cv_config': {
        'method': 'TimeSeriesSplit',
        'n_splits': N_SPLITS
    },
    'created': datetime.now().isoformat()
}

params_path = f"{MODEL_PATH}/optimal_hyperparameters.json"
with open(params_path, 'w') as f:
    json.dump(optimal_params, f, indent=2, default=str)

print(f"💾 Optimální parametry uloženy: {params_path}")

In [ ]:
# Uložení nejlepších modelů
best_reg_path = f"{MODEL_PATH}/fundamental_predictor_tuned.pkl"
best_clf_path = f"{MODEL_PATH}/price_classifier_tuned.pkl"

joblib.dump(grid_search_reg.best_estimator_, best_reg_path)
joblib.dump(grid_search_clf.best_estimator_, best_clf_path)

print(f"💾 Tuned Regressor: {best_reg_path}")
print(f"💾 Tuned Classifier: {best_clf_path}")

---
## 10. Shrnutí

### ✅ Dokončeno:

| Úkol | Status |
|------|--------|
| Grid Search pro RF Regressor | ✅ |
| Grid Search pro RF Classifier | ✅ |
| Porovnání s Gradient Boosting | ✅ |
| Vizualizace výsledků | ✅ |
| Uložení optimálních parametrů | ✅ |

### 📁 Vytvořené soubory:

| Soubor | Popis |
|--------|-------|
| `models/optimal_hyperparameters.json` | Nejlepší parametry |
| `models/fundamental_predictor_tuned.pkl` | Optimalizovaný regressor |
| `models/price_classifier_tuned.pkl` | Optimalizovaný classifier |

### ➡️ Další notebook:

**Notebook 06: Final Evaluation**
- Kompletní evaluace všech modelů
- Vizualizace výsledků pro diplomovou práci

In [ ]:
# Finální shrnutí
print("="*70)
print("📊 NOTEBOOK 05 - SHRNUTÍ HYPERPARAMETER TUNING")
print("="*70)

print(f"\n🌲 RF Regressor (Fundamental Predictor):")
print(f"   Best MAE: {-grid_search_reg.best_score_:.4f}")
print(f"   Best params: {grid_search_reg.best_params_}")

print(f"\n🌲 RF Classifier (Price Classifier):")
print(f"   Best F1: {grid_search_clf.best_score_:.4f}")
print(f"   Best params: {grid_search_clf.best_params_}")

print(f"\n🚀 Gradient Boosting Classifier:")
print(f"   Best F1: {grid_search_gb.best_score_:.4f}")
print(f"   Best params: {grid_search_gb.best_params_}")

print(f"\n✅ Hyperparameter tuning dokončen!")